In [111]:
import pandas as pd
pd.set_option('display.max_columns', None) 
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [112]:
input_path = 'H:\RediMinds\VCQI'
df_filter2_codes = pd.read_csv(input_path+"\VCQI_clean_pathology.csv")

In [113]:
# sepearte target variable from predictors for furthur processing
target = 'PATHOLOGY'
X = df_filter2_codes.drop(columns=[target])
Y = df_filter2_codes[target]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3084 entries, 0 to 3083
Data columns (total 34 columns):
PROC NAME                          3084 non-null int64
GENDER                             3084 non-null int64
AGE AT SURGERY                     3084 non-null float64
MARITAL STATUS                     3084 non-null int64
RACE                               3084 non-null int64
WEIGHT                             3084 non-null float64
HEIGHT                             3084 non-null float64
BMI                                3084 non-null float64
CLINICAL SIZE (mm)                 3084 non-null float64
SYMPTOMS                           3084 non-null int64
SOLITARY KIDNEY                    3084 non-null int64
BILATERALITY OF TUMOR              3084 non-null int64
SIDE OF TUMOR                      3084 non-null int64
SIDE OF SURGERY                    3084 non-null int64
FACE                               3084 non-null int64
TUMOR lOCATION                     3084 non-null int64
PRE

In [114]:
# Create dummy variables
with open (input_path+'\cat_col_pathalogy', 'rb') as fp:
    cat_col = pickle.load(fp)


nn_data = X.copy()
for col in cat_col:
    if not col == 'PATHOLOGY':
        nn_data = nn_data.merge(pd.get_dummies(X[col], prefix=col), how = 'left', left_index = True, right_index =True) # create dummy variables
        nn_data = nn_data.drop(labels = col, axis = 'columns') # drop original variables

In [115]:
nn_data.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,ASA SCORE_0,ASA SCORE_1,ASA SCORE_2,ASA SCORE_3,ASA SCORE_4,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,50.0,57.00,168.0,20.20,7.0,13.2,50.0,7800.0,0.63,100.030000,1.0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0
1,69.0,68.60,161.3,26.37,74.0,14.7,44.2,6400.0,0.80,71.119672,1.0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
2,83.0,83.65,164.0,31.10,80.0,12.5,38.9,5590.0,0.97,54.844682,1.0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
3,66.0,107.00,180.0,32.78,73.0,17.0,50.8,8000.0,0.93,81.290783,1.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
4,61.0,82.90,171.0,28.35,8.0,12.1,37.1,7710.0,1.10,68.053260,1.0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0


In [116]:
# Split data in test and train
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(nn_data, Y, test_size=0.20, random_state=42)

In [117]:
print("Number of records in trainset {}".format(len(x_train)))
print("Number records in testset {}".format(len(x_test)))

Number of records in trainset 2467
Number records in testset 617


In [118]:
# Standardize the numeric columns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
stdc = StandardScaler()

x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
float_col = x_train.select_dtypes('float').columns
x_train[float_col] = pd.DataFrame(stdc.fit_transform(x_train[float_col]),columns=float_col).copy()
x_test[float_col] = pd.DataFrame(stdc.transform(x_test[float_col]),columns=float_col).copy()

H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [119]:
# input layer
input_layer = keras.layers.Input(shape=(x_train.shape[1],), name = "input_layer")
x = keras.layers.Dense(256, name = 'Dense_1')(input_layer)
x = keras.layers.Dropout(0.2, name=  'Dropout_1')(x)
x = keras.layers.Dense(256, name = 'Dense_2')(x)
x = keras.layers.Dropout(0.2, name=  'Dropout_2')(x)
main_output = keras.layers.Dense(1, activation='sigmoid',name='main_output')(x)

model = keras.Model(inputs= input_layer, outputs=main_output)

# compiling the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve = 'ROC',name = 'AUC_ROC'),
                       tf.keras.metrics.AUC(curve = 'PR', name = 'AUC_PR')],
              )

# Keras callback. The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_AUC_ROC', patience=10, mode='max',restore_best_weights=False)


model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 102)]             0         
_________________________________________________________________
Dense_1 (Dense)              (None, 256)               26368     
_________________________________________________________________
Dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
Dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
Dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
main_output (Dense)          (None, 1)                 257       
Total params: 92,417
Trainable params: 92,417
Non-trainable params: 0
_______________________________________________________

In [120]:
model.fit(x = x_train, y = y_train,
          validation_split = 0.2,
          epochs = 50,
          callbacks = [early_stop]
         )


Train on 1973 samples, validate on 494 samples
Epoch 1/50
1973/1973 [==============================] - 1s 490us/sample - loss: 0.5642 - AUC_ROC: 0.5474 - AUC_PR: 0.2228 - val_loss: 0.4872 - val_AUC_ROC: 0.5710 - val_AUC_PR: 0.2186
Epoch 2/50
1973/1973 [==============================] - 0s 99us/sample - loss: 0.4840 - AUC_ROC: 0.6538 - AUC_PR: 0.3203 - val_loss: 0.4734 - val_AUC_ROC: 0.6364 - val_AUC_PR: 0.2701
Epoch 3/50
1973/1973 [==============================] - 0s 101us/sample - loss: 0.4742 - AUC_ROC: 0.6791 - AUC_PR: 0.3733 - val_loss: 0.4581 - val_AUC_ROC: 0.6749 - val_AUC_PR: 0.3204
Epoch 4/50
1973/1973 [==============================] - 0s 101us/sample - loss: 0.4673 - AUC_ROC: 0.6992 - AUC_PR: 0.3676 - val_loss: 0.4482 - val_AUC_ROC: 0.6975 - val_AUC_PR: 0.3493
Epoch 5/50
1973/1973 [==============================] - 0s 98us/sample - loss: 0.4603 - AUC_ROC: 0.7090 - AUC_PR: 0.4029 - val_loss: 0.4439 - val_AUC_ROC: 0.7106 - val_AUC_PR: 0.3709
Epoch 6/50
1973/1973 [=============

Epoch 45/50
1973/1973 [==============================] - 0s 97us/sample - loss: 0.4204 - AUC_ROC: 0.7730 - AUC_PR: 0.5183 - val_loss: 0.4262 - val_AUC_ROC: 0.7347 - val_AUC_PR: 0.4800
Epoch 46/50
1973/1973 [==============================] - 0s 105us/sample - loss: 0.4261 - AUC_ROC: 0.7679 - AUC_PR: 0.4942 - val_loss: 0.4285 - val_AUC_ROC: 0.7316 - val_AUC_PR: 0.4785
Epoch 47/50
1973/1973 [==============================] - 0s 99us/sample - loss: 0.4257 - AUC_ROC: 0.7677 - AUC_PR: 0.5098 - val_loss: 0.4260 - val_AUC_ROC: 0.7343 - val_AUC_PR: 0.4783
Epoch 48/50
1973/1973 [==============================] - 0s 102us/sample - loss: 0.4265 - AUC_ROC: 0.7703 - AUC_PR: 0.4856 - val_loss: 0.4243 - val_AUC_ROC: 0.7351 - val_AUC_PR: 0.4785
Epoch 49/50
1973/1973 [==============================] - 0s 98us/sample - loss: 0.4245 - AUC_ROC: 0.7705 - AUC_PR: 0.4974 - val_loss: 0.4237 - val_AUC_ROC: 0.7357 - val_AUC_PR: 0.4828
Epoch 50/50
1973/1973 [==============================] - 0s 100us/sample - los

In [121]:
# predict on test set
pd.DataFrame(model.predict(x_test))

,0
0,0.321250
1,0.124098
2,0.100972
3,0.406363
4,0.211189
5,0.614680
6,0.057687
7,0.165189
8,0.527871
9,0.097318


In [122]:
results_nn = pd.DataFrame(model.predict(x_test), columns=['pred_prob'])
results_nn['pred_label'] =  results_nn['pred_prob'].apply(lambda x: 1 if x>=0.5 else 0)
results_nn['true_label'] = np.array(y_test)

In [123]:
# NeuralNetwork Score Raw Data
print("\n Model Balanced Accuracy: \n" + str(metrics.balanced_accuracy_score(results_nn['true_label'], results_nn['pred_label'])))
print("\n Confusion Matrix : \n"+str(metrics.confusion_matrix(results_nn['true_label'], results_nn['pred_label'])))
print("\n Classification Report: \n"+ str(metrics.classification_report(results_nn['true_label'], results_nn['pred_label'])))
print("\n AUC-ROC: \n"+ str(metrics.roc_auc_score(results_nn['true_label'], results_nn['pred_prob'])))


def calc_aucpr_data(result):
    y_ACTUAL = result['true_label']
    scores_prob = result['pred_prob']
    yhat = result['pred_label']
    precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
    prc_auc = metrics.auc(recall,precision)
    return prc_auc

print("\n PR-AUC: \n"+ str(calc_aucpr_data(results_nn)))


 Model Balanced Accuracy: 
0.5526260162601626

 Confusion Matrix : 
[[469  23]
 [106  19]]

 Classification Report: 
              precision    recall  f1-score   support

           0       0.82      0.95      0.88       492
           1       0.45      0.15      0.23       125

    accuracy                           0.79       617
   macro avg       0.63      0.55      0.55       617
weighted avg       0.74      0.79      0.75       617


 AUC-ROC: 
0.6876422764227642

 PR-AUC: 
0.3531032218345356
